# Fixed-Length Few-Shot


In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {
        "question": "What is the capital of France?",
        "answer": "Paris"
    },
    {
        "question": "Who wrote the play 'Romeo and Juliet'?",
        "answer": "William Shakespeare"
    },
    {
        "question": "What is the chemical symbol for water?",
        "answer": "H2O"
    },
    {
        "question": "What is the largest planet in our solar system?",
        "answer": "Jupiter"
    },
    {
        "question": "What year did the Titanic sink?",
        "answer": "1912"
    },
    {
        "question": "What is the fastest land animal?",
        "answer": "Cheetah"
    },
    {
        "question": "Who was the first President of the United States?",
        "answer": "George Washington"
    },
    {
        "question": "What is the square root of 64?",
        "answer": "8"
    },
    {
        "question": "Which element has the atomic number 1?",
        "answer": "Hydrogen"
    },
    {
        "question": "What is the largest ocean on Earth?",
        "answer": "Pacific Ocean"
    },
    {
        "question": "What is the smallest country in the world?",
        "answer": "Vatican City"
    },
    {
        "question": "Who painted the Mona Lisa?",
        "answer": "Leonardo da Vinci"
    }
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}")
    ]
)

In [2]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.format())

Human: What is the capital of France?
AI: Paris
Human: Who wrote the play 'Romeo and Juliet'?
AI: William Shakespeare
Human: What is the chemical symbol for water?
AI: H2O
Human: What is the largest planet in our solar system?
AI: Jupiter
Human: What year did the Titanic sink?
AI: 1912
Human: What is the fastest land animal?
AI: Cheetah
Human: Who was the first President of the United States?
AI: George Washington
Human: What is the square root of 64?
AI: 8
Human: Which element has the atomic number 1?
AI: Hydrogen
Human: What is the largest ocean on Earth?
AI: Pacific Ocean
Human: What is the smallest country in the world?
AI: Vatican City
Human: Who painted the Mona Lisa?
AI: Leonardo da Vinci


In [3]:
from langchain_core.output_parsers import StrOutputParser
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You only need to return a single answer without any explanation"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

model = ChatOpenAI()

chain = final_prompt | model | StrOutputParser()

response = chain.invoke({
    "input": "What is the capital of Malaysia?"
})

print(response)

Kuala Lumpur
